In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-05-10 13:00:00+00:00


In [51]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [33]:
import hopsworks
import src.config as config
from hsfs.client.exceptions import RestAPIError

# Step 1: Login to Hopsworks and get feature store
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY,
)
fs = project.get_feature_store()

# Step 2: Try fetching the feature group
try:
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)
    print("✅ Feature group found.")
except RestAPIError:
    print("❌ Feature group not found. Check name or create it first.")
    fg = None


2025-05-10 08:01:24,764 INFO: Initializing external client
2025-05-10 08:01:24,765 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 08:01:25,590 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1228960
✅ Feature group found.


In [39]:
# 17_fetch_predictions.ipynb

import pandas as pd
import hopsworks
import src.config as config

print("🔍 Looking for Feature Group:")
print("Name:", config.FEATURE_GROUP_MODEL_PREDICTION)
print("Version:", config.FEATURE_GROUP_MODEL_PREDICTION_VERSION)

# Step 1: Login to Hopsworks
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY,
)
fs = project.get_feature_store()
print("✅ Connected to Hopsworks.")

# Step 2: Access the predictions Feature Group
fg = fs.get_feature_group(name="citi_bike_prediction", version=2)


# Step 3: Read recent prediction data
df = fg.read()
df["pickup_hour"] = pd.to_datetime(df["pickup_hour"])
df["pickup_location_id"] = df["pickup_location_id"].astype(str)
df["predicted_rides"] = df["predicted_rides"].astype(int)

latest_hour = df["pickup_hour"].max()
recent_preds = df[df["pickup_hour"] == latest_hour].sort_values("predicted_rides", ascending=False)

import matplotlib.pyplot as plt

# Optional: Quick bar plot
recent_preds.plot(
    x="pickup_location_id", 
    y="predicted_rides", 
    kind="bar", 
    title=f"Predicted Citi Bike Rides for {latest_hour}", 
    legend=False
)
plt.ylabel("Rides")
plt.xlabel("Location ID")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Display top rows
recent_preds.head()

🔍 Looking for Feature Group:
Name: citi_bike_prediction
Version: 2
2025-05-10 08:06:18,670 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 08:06:18,673 INFO: Initializing external client
2025-05-10 08:06:18,674 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 08:06:19,444 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1228960
✅ Connected to Hopsworks.


AttributeError: 'NoneType' object has no attribute 'read'

In [57]:
fg = fs.get_feature_group(name="citi_bike_prediction", version=2)


In [71]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [75]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-05-10 08:12:05,258 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-10 08:12:05,261 INFO: Initializing external client
2025-05-10 08:12:05,261 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-10 08:12:06,008 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1228960


In [81]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-05-10 13:00:00+00:00
